# Implementing Recommendation Engines with Matching Engine

![](img/arch.png)

#### VPC Network peering
Matching engine is a high performance vector matching service that requires a seperate VPC to ensure performance. 

Below are the one-time instructions to set up a peering network. 

# **Once created, be sure to your notebook instance running this particular notebook is in the subnetwork... https://cloud.google.com/vertex-ai/docs/matching-engine/match-eng-setup **

![](img/subnetwork2.png) 
Then select the network from advanced options
![](img/subnetwork.png)

In [1]:
# PROJECT_ID = "hybrid-vertex" 
# NETWORK_NAME = "ucaip-haystack-vpc-network"  
# PEERING_RANGE_NAME = "ucaip-haystack-range"

# # Create a VPC network
# ! gcloud compute networks create {NETWORK_NAME} --bgp-routing-mode=regional --subnet-mode=auto --project={PROJECT_ID}

# # Add necessary firewall rules
# ! gcloud compute firewall-rules create {NETWORK_NAME}-allow-icmp --network {NETWORK_NAME} --priority 65534 --project {PROJECT_ID} --allow icmp

# ! gcloud compute firewall-rules create {NETWORK_NAME}-allow-internal --network {NETWORK_NAME} --priority 65534 --project {PROJECT_ID} --allow all --source-ranges 10.128.0.0/9

# ! gcloud compute firewall-rules create {NETWORK_NAME}-allow-rdp --network {NETWORK_NAME} --priority 65534 --project {PROJECT_ID} --allow tcp:3389

# ! gcloud compute firewall-rules create {NETWORK_NAME}-allow-ssh --network {NETWORK_NAME} --priority 65534 --project {PROJECT_ID} --allow tcp:22

# # Reserve IP range
# ! gcloud compute addresses create {PEERING_RANGE_NAME} --global --prefix-length=16 --network={NETWORK_NAME} --purpose=VPC_PEERING --project={PROJECT_ID} --description="peering range for uCAIP Haystack."

# Set up peering with service networking
# ! gcloud services vpc-peerings connect --service=servicenetworking.googleapis.com --network=${NETWORK_NAME} --ranges=${PEERING_RANGE_NAME} --project=${PROJECT_ID}

### Deploy Playlist Query Model to a Vertex endpoint
This will be the endpoint that a user will query with thier last n songs played 

In [1]:
import os

PROJECT_ID = 'hybrid-vertex'  # <--- TODO: CHANGE THIS
LOCATION = 'us-central1' 
DIMENSIONS = 128 # must match output dimensions - embedding dim in two tower code
DISPLAY_NAME = "spotify_merlin_candidates_v1"
BUCKET = 'gs://spotify-beam-v3'
OUTPUT_PATH = os.path.join(BUCKET, "merlin-processed")
MODEL_PATH = os.path.join(OUTPUT_PATH, 'query_model_merlin')

import sys
from google.cloud import aiplatform_v1beta1 #needed for matching engine calls


from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

EMBEDDINGS_INITIAL_URI = os.path.join(OUTPUT_PATH, 'embeddings')

### Create a matching engine index

The matching engine loads an index from a file of embeddings created from the last notebook. 

Many of the optimization options for matching engine are found in the ah tree settings and testing is recommended depending on each use case

Recall we saved our two tower models and query embeddings (newline json) in a candidate folder like so:

![](img/saved-models.png)

## Set the Nearest Neighbor Options

See here for tips on [tuning the index](https://cloud.google.com/vertex-ai/docs/matching-engine/using-matching-engine#tuning_the_index)

From the paper - here's the rough idea

1. (Initialization Step) Select a dictionary C(m) bysampling from {x(m) 1 , . . . x (m) n }. 
2. (Partition Assignment Step) For each datapoint xi , update x˜i by using the value of c ∈ C (m) that minimizes the anisotropic loss of ˜xi.
3. (Codebook Update Step) Optimize the loss function over all codewords in all dictionaries while keeping every dictionaries partitions constant.
4. Repeat Step 2 and Step 3 until convergence to a fixed point or maximum number of iteration is reached.


### Relating the algorithm to the parameters:

* `leafNodeEmbeddingCount` -> Number of embeddings on each leaf node. The default value is 1000 if not set.
* `leafNodesToSearchPercent` -> The default percentage of leaf nodes that any query may be searched. Must be in range 1-100, inclusive. The default value is 10 (means 10%) if not set.
* `approximateNeighborsCount` -> The default number of neighbors to find through approximate search before exact reordering is performed. Exact reordering is a procedure where results returned by an approximate search algorithm are reordered via a more expensive distance computation.
* `distanceMeasureType` -> DOT_PRODUCT_DISTANCE is default - COSINE, L1 and L2^2 is available

Other best practices from our PM team:
```
Start from leafNodesToSearchPercent=5 and approximateNeighborsCount=10 * k

use default values for others.

measure performance and recall and change those 2 parameters accordingly.
```

In [3]:
!gsutil ls gs://spotify-beam-v3/merlin-processed

gs://spotify-beam-v3/merlin-processed/embeddings/
gs://spotify-beam-v3/merlin-processed/full_two_tower_model_merlin/
gs://spotify-beam-v3/merlin-processed/query_model_merlin/
gs://spotify-beam-v3/merlin-processed/tb_logs/
gs://spotify-beam-v3/merlin-processed/train/
gs://spotify-beam-v3/merlin-processed/valid/
gs://spotify-beam-v3/merlin-processed/workflow/


In [ ]:
tree_ah_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=DISPLAY_NAME,
    contents_delta_uri=EMBEDDINGS_INITIAL_URI,
    dimensions=DIMENSIONS,
    approximate_neighbors_count=50,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    leaf_node_embedding_count=500,
    leaf_nodes_to_search_percent=7,
    description="Songs embeddings from the Spotify million playlist dataset",
    labels={"label_name": "label_value"},
)

Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/934903580331/locations/us-central1/indexes/6862905289973170176/operations/491122500910972928


## This takes 20-30 minutes - here's some reading on what it is doing
#### Note on the advantages of the algorithm

[link](https://arxiv.org/pdf/1908.10396.pdf)

```However, it is easy to see that not all pairs of (x, q) are equally important. The approximation error on the pairs which have a high inner product is far more important since they are likely to be among the top ranked pairs and can greatly affect the search result, while for the pairs whose inner product is low the approximation error matters much less. In other words, for a given datapoint x, we should quantize it with a bigger focus on its error with those queries which have high inner product with x. See Figure 1 for the illustration.```


![](img/algo.png)


### Create a matching engine endpoint

Below we set the variable names for the endpoint along with other key values for resource creation

In [ ]:
ENDPOINT = "{}-aiplatform.googleapis.com".format(LOCATION)
NETWORK_NAME = "ucaip-haystack-vpc-network"  # @param {type:"string"}
PROJECT_NUMBER = !gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = PROJECT_NUMBER[0]

PARENT = "projects/{}/locations/{}".format(PROJECT_ID, LOCATION)

print("ENDPOINT: {}".format(ENDPOINT))
print("PROJECT_ID: {}".format(PROJECT_ID))
print("REGION: {}".format(LOCATION))

!gcloud config set project {PROJECT_ID}
!gcloud config set ai_platform/region {LOCATION}

### **If you start getting errors about consumers, ensure this notebook is running in the VPC network created**

In [ ]:
index_endpoint_client = aiplatform_v1beta1.IndexEndpointServiceClient(
    client_options=dict(api_endpoint=ENDPOINT)
)


VPC_NETWORK_NAME = "projects/{}/global/networks/{}".format(PROJECT_NUMBER, NETWORK_NAME) #FQN for VPC network name

index_client = {
    "display_name": "index_endpoint_for_demo_merlin",
    "network": VPC_NETWORK_NAME,
}

r = index_endpoint_client.create_index_endpoint(
    parent=PARENT, index_endpoint=index_client
)

## Other quick notes on ME while we wait for deployment

[link](https://cloud.google.com/blog/topics/developers-practitioners/find-anything-blazingly-fast-googles-vector-search-technology)

Instead of comparing vectors one by one, you could use the approximate nearest neighbor (ANN) approach to improve search times. Many ANN algorithms use vector quantization (VQ), in which you split the vector space into multiple groups, define "codewords" to represent each group, and search only for those codewords. This VQ technique dramatically enhances query speeds and is the essential part of many ANN algorithms, just like indexing is the essential part of relational databases and full-text search engines.

![](img/vectorQuant.gif)


As you may be able to conclude from the diagram above, as the number of groups in the space increases the speed of the search decreases and the accuracy increases.  Managing this trade-off — getting higher accuracy at shorter latency — has been a key challenge with ANN algorithms. 

Last year, Google Research announced ScaNN, a new solution that provides state-of-the-art results for this challenge. With ScaNN, they introduced a new VQ algorithm called anisotropic vector quantization:

![](img/Loss_Types.max-1000x1000.png)

Anisotropic vector quantization uses a new loss function to train a model for VQ for an optimal grouping to capture farther data points (i.e. higher inner product) in a single group. With this idea, the new algorithm gives you higher accuracy at lower latency, as you can see in the benchmark result below (the violet line): 

![](img/speedvsaccuracy.max-1600x1600.png)


In [ ]:
INDEX_ENDPOINT_NAME = r.result().name
INDEX_ENDPOINT_NAME

In [ ]:
INDEX_ENDPOINT_NAME

### Deploy the index to the endpoint and create and endpoint object

In [ ]:
DEPLOYED_INDEX_ID = "deployed_merlin_spotify_v1"

deploy_ann_index = {
    "id": DEPLOYED_INDEX_ID,
    "display_name": DEPLOYED_INDEX_ID,
    "index": tree_ah_index.resource_name,
}

s = index_endpoint_client.deploy_index(
    index_endpoint=INDEX_ENDPOINT_NAME, deployed_index=deploy_ann_index
)

In [ ]:
# Poll the operation until it's done successfullly.
import time

while True:
    if s.done():
        break
    print("Poll the operation to deploy index...")
    time.sleep(60)
s.result()

In [ ]:
# finally create the endpoint object
ME_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(INDEX_ENDPOINT_NAME)
ME_index_endpoint

### You should now see matching engine resources in your GCP console:

![](img/me-resources2.png)

![](img/me-indexes.png)

## Note the endpoint has already been created in the `02-build-custom-query-predictor`


In [2]:
#from the last noteobook copy/paste

endpoint = aiplatform.Endpoint('projects/934903580331/locations/us-central1/endpoints/7569218365516939264')


## Putting it together: Combine the query endpoint with the matching engine endpoint for state-of the art recommendations

You can grab a quick example from training or build yourself by utilizing the returned example structure. Skip is to get to a psuedo-random example. The data is the same from training - you can also construct your own test instances using the format provided


```python
for tensor_dict in train_dataset.unbatch().skip(12905).take(1):
    td_keys = tensor_dict.keys()
    list_dict = {}
    for k in td_keys:
        list_dict.update({k: tensor_dict[k].numpy()})
    print(list_dict)
```

## Get some test instances and run through the endpoint and matching engine

In [17]:
list(TEST_INSTANCE.keys())

['collaborative',
 'album_name_pl',
 'artist_genres_pl',
 'artist_name_pl',
 'artist_pop_can',
 'description_pl',
 'duration_ms_songs_pl',
 'n_songs_pl',
 'name',
 'num_albums_pl',
 'num_artists_pl',
 'track_name_pl',
 'track_pop_pl',
 'duration_ms_seed_pl',
 'pid',
 'track_uri_pl']

In [ ]:
## Ground truth candidate:
    # 'album_uri_can': 'spotify:album:5l83t3mbVgCrIe1VU9uJZR', 
    # 'artist_name_can': 'Russ', 
    # 'track_name_can': 'We Just Havent Met Yet', 
## TODO - we have to overload with candidate data because of the workflow transform, add overloaded values in the predictor
TEST_INSTANCE = {'collaborative': 'false',
                 'album_name_pl': ["There's Really A Wolf", 'Late Nights: The Album',
                       'American Teen', 'Crazy In Love', 'Pony'], 
                 'artist_genres_pl': ["'hawaiian hip hop', 'rap'",
                       "'chicago rap', 'dance pop', 'pop', 'pop rap', 'r&b', 'southern hip hop', 'trap', 'urban contemporary'",
                       "'pop', 'pop r&b'", "'dance pop', 'pop', 'r&b'",
                       "'chill r&b', 'pop', 'pop r&b', 'r&b', 'urban contemporary'"], 
                 'artist_name_pl': ['Russ', 'Jeremih', 'Khalid', 'Beyonc\xc3\xa9',
                       'William Singe'], 
                 'artist_pop_can': 82.0, 
                 'description_pl': '', 
                 'duration_ms_songs_pl': [237506.0, 217200.0, 219080.0, 226400.0, 121739.0], 
                 'n_songs_pl': 8.0, 
                 'name': 'Lit Tunes ', 
                 'num_albums_pl': 8.0, 
                 'num_artists_pl': 8.0, 
                 'track_name_pl': ['Losin Control', 'Paradise', 'Location',
                       'Crazy In Love - Remix', 'Pony'], 
                 'track_pop_pl': [79.0, 58.0, 83.0, 71.0, 57.0],
                 'duration_ms_seed_pl': 51023.1,
                 'pid': 1,
                 'track_uri_pl': ['spotify:track:4cxMGhkinTocPSVVKWIw0d',
                       'spotify:track:1wNEBPo3nsbGCZRryI832I',
                       'spotify:track:152lZdxL1OR0ZMW6KquMif',
                       'spotify:track:2f4IuijXLxYOeBncS60GUD',
                       'spotify:track:4Lj8paMFwyKTGfILLELVxt']
                     }

#### Before we submit the prediction to test, we have to call the workflow
Ideally this could be implemented with a custom prediction routine so the data is pre-processed ahead of time.

More info on how to implement a pre-processer custom model: https://cloud.google.com/vertex-ai/docs/predictions/custom-prediction-routines

# Get our predictions for the next song recommendation - now let's feed the embedding to find the closest neighbors
(higher distance score better - this is inner product)

In [ ]:
playlist_emb = endpoint.predict([TEST_INSTANCE])

In [14]:
ME_index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/934903580331/locations/us-central1/indexEndpoints/308478982288834560')

In [15]:
ME_index_endpoint.match(deployed_index_id='deployed_merlin_spotify_v1',
                       queries=playlist_emb.predictions,
                       num_neighbors=10)

[[MatchNeighbor(id='spotify:track:6rvjjyTCbuoIucuI9TvTfX', distance=118.5457992553711),
  MatchNeighbor(id='spotify:track:2jNFga2ldTWgXXuu57u6QB', distance=117.87425994873047),
  MatchNeighbor(id='spotify:track:5RINSPsc11CjSj9OhBbHeV', distance=117.82361602783203),
  MatchNeighbor(id='spotify:track:14yaVE5zyeIZ5OopvQFVtG', distance=117.8228530883789),
  MatchNeighbor(id='spotify:track:5sDKFCwJsnIS31t3IaCoZt', distance=117.7808609008789),
  MatchNeighbor(id='spotify:track:3vW77S4JolRGbunmXuJMIV', distance=117.58699035644531),
  MatchNeighbor(id='spotify:track:3gY3fTkff4VmZ2XfvU6N4l', distance=117.57185363769531),
  MatchNeighbor(id='spotify:track:1uE9BkKQtUT03iPPk55oKO', distance=117.54019927978516),
  MatchNeighbor(id='spotify:track:2csIyFgOxMfxE9yn02Rc2A', distance=117.40080261230469),
  MatchNeighbor(id='spotify:track:5bxRZTJbd2vApE4ZyakL0h', distance=117.31294250488281)]]

# Here's our Recommendations!

  ### No Scurbs
`[[MatchNeighbor(id='spotify:track:6rvjjyTCbuoIucuI9TvTfX', distance=118.5457992553711),`

![](img/spotify/1.png)
_______

  ### I Know
`MatchNeighbor(id='spotify:track:2jNFga2ldTWgXXuu57u6QB', distance=117.87425994873047),`

![](img/spotify/2.png)

_______

  ### Eye 2 Eye
`MatchNeighbor(id='spotify:track:5RINSPsc11CjSj9OhBbHeV', distance=117.82361602783203),`

![](img/spotify/3.png)

_______

  ### Just a Friend
`MatchNeighbor(id='spotify:track:14yaVE5zyeIZ5OopvQFVtG', distance=117.8228530883789),`

![](img/spotify/3.png)


_______

  ### Mirror 
`MatchNeighbor(id='spotify:track:5sDKFCwJsnIS31t3IaCoZt', distance=117.7808609008789),`

![](img/spotify/4.png)


_______

   ### Hello
`MatchNeighbor(id='spotify:track:3vW77S4JolRGbunmXuJMIV', distance=117.58699035644531),`

![](img/spotify/5.png)


_______

  ### home
`MatchNeighbor(id='spotify:track:3gY3fTkff4VmZ2XfvU6N4l', distance=117.57185363769531),`

![](img/spotify/6.png)


_______

  ### Undercover - Coucheron Remix
`MatchNeighbor(id='spotify:track:1uE9BkKQtUT03iPPk55oKO', distance=117.54019927978516),`

![](img/spotify/7.png)

_______

  ### Undercover - Devault Remix

`MatchNeighbor(id='spotify:track:2csIyFgOxMfxE9yn02Rc2A', distance=117.40080261230469),`




![](img/spotify/7.png)

_______

### Got That Bomb
`MatchNeighbor(id='spotify:track:5bxRZTJbd2vApE4ZyakL0h', distance=117.31294250488281)]]`

![](img/spotify/3.png)


### One more test with another playlist

It is easy to explore and find another example with a query like this, then export the result to json

```sql
SELECT
  collaborative,
  album_name_pl,
  artist_genres_pl,
  artist_name_pl,
  artist_pop_can,
  description_pl,
  duration_ms_songs_pl,
  n_songs_pl,
  name,
  num_albums_pl,
  num_artists_pl,
  track_name_pl,
  track_pop_pl,
  duration_ms_seed_pl,
  pid,
  track_uri_pl
FROM
  `hybrid-vertex.mdp_eda_test.train_flatten_last_5`
WHERE
  pid_pos_id ='47626-22'
  AND artist_name_can = 'Radiohead'
LIMIT
  1000
  ```

In [29]:
INSTANCE2 = {"collaborative":"false",
             "album_name_pl":["Pinkerton","Murmur","Monster","New Adventures In Hi-Fi","Vs."],
             "artist_genres_pl":["'alternative rock', 'modern power pop', 'modern rock', 'permanent wave', 'rock'","'alternative rock', 'athens indie', 'classic rock', 'permanent wave', 'pop rock', 'rock'","'alternative rock', 'athens indie', 'classic rock', 'permanent wave', 'pop rock', 'rock'","'alternative rock', 'athens indie', 'classic rock', 'permanent wave', 'pop rock', 'rock'","'grunge', 'permanent wave', 'rock'"],"artist_name_pl":["Weezer","R.E.M.","R.E.M.","R.E.M.","Pearl Jam"],
             "artist_pop_can":82,
             "description_pl":"",
             "duration_ms_songs_pl":[238468,245200,240400,212026,188586],
             "n_songs_pl":22,"name":"Road Trip",
             "num_albums_pl":21,
             "num_artists_pl":17,
             "track_name_pl":["Pink Triangle","Radio Free Europe","What's The Frequency, Kenneth?","Low Desert","Leash (Remastered)"],
             "track_pop_pl":[60,49,0,0,47],
             "duration_ms_seed_pl":5507859,
             "pid":47626, #be sure to unquote this - bq thinks this is a string?
             "track_uri_pl":["spotify:track:6qQGxKyy6LcyZVsWn93lyS","spotify:track:46dGFTD918NMz1IP1rPJXO","spotify:track:1yStBw3HI2pnjbO3ihq33N","spotify:track:2xRdlHxMjokYAVQ0nvcUM7","spotify:track:52FlqmFHKaTV0dDMEe0F4F"]}

In [31]:
ME_index_endpoint.match(deployed_index_id='deployed_merlin_spotify_v1',
                       queries=endpoint.predict([INSTANCE2]).predictions,
                       num_neighbors=10)

[[MatchNeighbor(id='spotify:track:2Sw1lJWQP4a074bjdy3eUH', distance=130.5625),
  MatchNeighbor(id='spotify:track:0ASBGQASJpCkjKFNCfX6MV', distance=130.5594482421875),
  MatchNeighbor(id='spotify:track:17nrjNJJL4ujTHM8ia4Oxf', distance=130.52471923828125),
  MatchNeighbor(id='spotify:track:5y4GrsURgB4jFWhd6u5Fgt', distance=130.50601196289062),
  MatchNeighbor(id='spotify:track:6OnND3qgACgRiXB6R8hWmE', distance=130.4535675048828),
  MatchNeighbor(id='spotify:track:4A9QsDKQox1rfFBNNo6t4C', distance=130.4269561767578),
  MatchNeighbor(id='spotify:track:0bE5gJJno8d6jBLgqHmx7k', distance=130.34982299804688),
  MatchNeighbor(id='spotify:track:7GEQic2y8KjVONDyRvZgCK', distance=130.32960510253906),
  MatchNeighbor(id='spotify:track:6Kt22eY25Ea5rUU2Fi36fy', distance=130.31768798828125),
  MatchNeighbor(id='spotify:track:0uIylCclPvfhDuyejkSfpC', distance=130.3046417236328)]]

#### All set - you now have a working recommendation system!
See the next notebook if you want to explore the system and make recommendations for yourself

In [ ]:
# ## Cleanup
# from google.cloud import aiplatform_v1beta1 #needed for matching engine calls

# REGION = 'us-central1'
# ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# index_client = aiplatform_v1beta1.IndexServiceClient(
#     client_options=dict(api_endpoint=ENDPOINT)
# )

# index_client.delete_index(name='deployed_spotify_v1')
# index_client.delete_index(name=INDEX_BRUTE_FORCE_RESOURCE_NAME)
# index_endpoint_client.delete_index_endpoint(name=INDEX_ENDPOINT_NAME)